# ML model development and Deployment using azureML (without mlflow)

When you run a script as an Azure Machine Learning job, you need to define the execution context for the job run. One key configuration is the compute target on which the script will be run. This could be the local workstation (in this case the compute instance), or a remote compute target such as the Azure Machine Learning managed compute cluster that is provisioned on-demand.

In this notebook, you'll create a compute cluster and explore compute targets for jobs.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [33]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.6.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\users\anshu\appdata\roaming\python\python310\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. The resource group name and workspace name are already filled in for you. You only need the subscription ID to complete the command.

To find the necessary parameters, click on the subscription and workspace name at the top right of the Studio. A pane will open on the right.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copy the subscription ID and replace **YOUR-SUBSCRIPTION-ID** with the value you copied. </p>

In [19]:
# enter details of your AML workspace
subscription_id = "4980acc2-d47d-47e6-8498-a6b0744658d8"
resource_group = "EYMAY"
workspace = "aml-workspace"

In [20]:
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential
credential = InteractiveBrowserCredential(tenant_id="13a86542-2185-4187-8e07-7512f5525c55")

In [21]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

In [22]:
from azure.ai.ml.entities import Workspace
try:
    ws = ml_client.workspace.get(workspace)
    print("workspace already exists")

except:
    ws = Workspace(name=workspace,location="centralindia")
    ws = ml_client.workspaces.begin_create(ws)

Class ManagedNetwork: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
The deployment request aml-workspace-276305 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F4980acc2-d47d-47e6-8498-a6b0744658d8%2FresourceGroups%2FEYMAY%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Faml-workspace-276305


## Create a compute cluster

In many cases, your local compute resources may not be sufficient to process a complex or long-running experiment that needs to process a large volume of data; and you may want to take advantage of the ability to dynamically create and use compute resources in the cloud. Azure Machine Learning supports a range of compute targets, which you can define in your workpace and use to run jobs; paying for the resources only when using them.

You can create a compute cluster in [Azure Machine Learning studio](https://ml.azure.com), by using the Python SDK, or the Azure CLI. The following code cell checks your workspace for the existence of a compute cluster names `aml-cluster`, and if it doesn't exist, creates it.

In [23]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS11_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=1,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=120,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)


Creating a new cpu compute target...


After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [24]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="cpu-cluster",
    max_instances=2
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [25]:
cpu_cluster = ml_client.compute.get("cpu-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

AMLCompute with name cpu-cluster has a maximum of 2 nodes


## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [26]:
import os
os.makedirs("src",exist_ok=True)

In [45]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)



# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

import os
import joblib

os.makedirs('outputs', exist_ok=True)
#file = open("outputs/version.txt","w")
#file.write(str(joblib.__version__))
#file.close()
joblib.dump(model, open("outputs/model.pkl","wb"))


Overwriting src/diabetes-training.py


## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [46]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (0.53 MBs): 100%|##########| 530144/530144 [00:00<00:00, 1220313.17it/s]




Monitor your job at https://ml.azure.com/runs/gifted_drop_6kqs1qj856?wsid=/subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/resourcegroups/EYMAY/workspaces/aml-workspace&tid=13a86542-2185-4187-8e07-7512f5525c55


<h4 style="background-color:Tomato;">wait till the job runs successfully, </h4>

In [47]:
print(returned_job.display_name,returned_job,returned_job.name)

diabetes-train-cluster type: command
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.gifted_drop_6kqs1qj856
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
resources:
  instance_count: 1
  shm_size: 2g
properties:
  mlflow.source.git.repoURL: https://github.com/anshupandey/MLOps-with-Azure
  mlflow.source.git.branch: main
  mlflow.source.git.commit: 74a2b02c1726d10446088b0b76cc6223498d5708
  azureml.git.dirty: 'True'
  _azureml.ComputeTargetType: amlctrain
  ContentSnapshotId: 1c6f2afd-ad63-4153-8aac-1902b34a27b3
compute: azureml:cpu-cluster
component:
  name: gifted_drop_6kqs1qj856
  display_name: diabetes-train-cluster
  type: command
  outputs:
    default:
      type: uri_folder
      mode: rw_mount
  command: python diabetes-training.py
  environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
  code: azureml:/subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/r


    A path on your local computer	                    mlflow-model/model.pkl
    A path on an Azure Machine Learning Datastore	    azureml://datastores/<datastore-name>/paths/<path_on_datastore>
    A path from an Azure Machine Learning job	        azureml://jobs/<job-name>/outputs/<output-name>/paths/<path-to-model-relative-to-the-named-output-location>
    A path from an MLflow job	                        runs:/<run-id>/<path-to-model-relative-to-the-root-of-the-artifact-location>
    A path from a Model Asset in Azure Machine Learning Workspace	azureml:<model-name>:<version>
    A path from a Model Asset in Azure Machine Learning Registry	azureml://registries/<registry-name>/models/<model-name>/versions/<version>

In [48]:
# register model
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
path = f"azureml://jobs/{returned_job.name}/outputs/artifacts/paths/outputs"

file_model = Model(path=path,type=AssetTypes.CUSTOM_MODEL,
                   name="diabetes_model")

out = ml_client.models.create_or_update(file_model)
out.path

'azureml://subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/resourceGroups/EYMAY/workspaces/aml-workspace/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.gifted_drop_6kqs1qj856/outputs'

In [49]:
out.name

'diabetes_model'

In [50]:
out.version

'2'

# Deployment: Real time online deployment

In [36]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

ep_name = "endpoint-diabetes" 
ep = ManagedOnlineEndpoint(name = ep_name,
                           auth_mode="key")
ep_result = ml_client.online_endpoints.begin_create_or_update(ep).result()

In [37]:
ep = ml_client.online_endpoints.get(name="endpoint-diabetes")

In [38]:
import os
os.makedirs("score",exist_ok=True)

In [51]:
%%writefile score/score.py

import os
import joblib
import json 
import logging
import numpy
def init():
    global model
    path = os.path.join(os.getenv("AZUREML_MODEL_DIR"),"outputs/model.pkl") # AZUREML_MODEL_DIR = ./azureml-models/MODEL_NAME/VERSION
    #logging.info(path)
    model = joblib.load(path)
    logging.info("initialization completed")

def run(raw_data):
    logging.info("model: request received")
    data = json.loads(raw_data)["data"]
    data = numpy.array(data)
    result = model.predict(data)
    logging.info("request procsesed")
    return result.tolist()

Overwriting score/score.py


In [52]:
%%writefile score/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - numpy
  - pandas
  - pip:
    - azureml-defaults==1.47.0
    - applicationinsights
    - inference-schema[numpy-support]==1.5
    - joblib==1.2.0

Overwriting score/conda.yml


In [54]:
# deployment configuration
from azure.ai.ml.entities import Model,ManagedOnlineDeployment,CodeConfiguration
from azure.ai.ml.constants import AssetTypes

# configure an environment
from azure.ai.ml.entities import Environment
env = Environment(
    conda_file="score/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220708.v1",
)

# configure an inference configuration with a scoring script
from azure.ai.ml.entities import CodeConfiguration
code_config = CodeConfiguration(
        code="score", scoring_script="score.py"
    )

model = ml_client.models.get(name=out.name, version=out.version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=ep_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
    environment=env,
    code_configuration=code_config
)

In [55]:
blue_deploy_result = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpoint-diabetes exists
Uploading score (0.0 MBs): 100%|##########| 887/887 [00:00<00:00, 3584.09it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...................................................................................................

In [56]:
# blue deployment takes 100 traffic
ep.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(ep)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


#### Check the status of the endpoint
You can check the status of the endpoint to see whether the model was deployed without error:


In [57]:

# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=ep_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)


Name: endpoint-diabetes
Status: Updating
Description: None


In [58]:
# existing traffic details
print(endpoint.traffic)

{'blue': 0}


### Inference

In [56]:
os.makedirs("testing",exist_ok=True)

In [59]:
%%writefile testing/sample-request.json
{"data": [
 [9,103,78,25,304,29.58219193,1.282869847,43], 
 [7,115,47,52,35,41.51152348,0.079018568,23]
]}

Overwriting testing/sample-request.json


In [60]:
# using json file to send a request using ml client
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=ep_name,
    deployment_name="blue",
    request_file="testing/sample-request.json",
)

'[1, 0]'

In [61]:
print("Scoring URI:", ep.scoring_uri)

Scoring URI: https://endpoint-diabetes.centralindia.inference.ml.azure.com/score


In [70]:
endpoint_cred = ml_client.online_endpoints.get_keys(name=ep_name).primary_key
endpoint_cred

'2dnsCMdcS4f5YrPLXu982f2go0b61DZP'

In [71]:
# sample testing via rest API
import json
import requests

url = ep.scoring_uri
api_key = endpoint_cred
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }
data = {"data": [
 [9,103,78,25,304,29.58219193,1.282869847,43], 
 [7,115,47,52,35,41.51152348,0.079018568,23]
]}

response = requests.post(url,data=json.dumps(data),headers=headers)
response.content

b'[1, 0]'

In [72]:
# delete the endpoint and deployment
ml_client.online_endpoints.begin_delete(name=ep_name)

.................................................................

## Batch Deployment

In [73]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data

import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = "diabetes-" + endpoint_suffix

In [74]:
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A classifier for batch inference",
)

In [75]:
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

In [76]:
compute_name = "cpu-cluster"

In [77]:
os.makedirs("batch_scoring_script",exist_ok=True)

Best practices for batch deployment : https://learn.microsoft.com/en-us/azure/machine-learning/how-to-batch-scoring-script?view=azureml-api-2&tabs=cli

In [78]:
%%writefile batch_scoring_script/batch_driver.py
import os
import pandas as pd
import joblib
import logging
import numpy as np


def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment
    # It is the path to the model folder
    path = os.path.join(os.getenv("AZUREML_MODEL_DIR"),"outputs/model.pkl") # AZUREML_MODEL_DIR = ./azureml-models/MODEL_NAME/VERSION
    model = joblib.load(path)
    logging.info("initialization completed")

def run(mini_batch) -> pd.DataFrame:
    print(f"Executing run method over batch of {len(mini_batch)} files.")

    results = []
    for data_batch in mini_batch:
        logging.info(data_batch)
        print(data_path)
        # Read comma-delimited data into an array
        data = np.genfromtxt(data_batch, delimiter=',')
        logging.info(data)
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append({"predictions":prediction})

    return pd.DataFrame(results)


Writing batch_scoring_script/batch_driver.py


In [79]:
# configure an inference configuration with a scoring script
from azure.ai.ml.entities import CodeConfiguration
code_config = CodeConfiguration(
        code="batch_scoring_script", scoring_script="batch_driver.py"
    )

In [80]:
from azure.ai.ml.constants import BatchDeploymentOutputAction
from azure.ai.ml.entities import BatchRetrySettings

deployment = BatchDeployment(
    name="diabetes-prediction",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute=compute_name,
    instance_count=1,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
    environment=env,
    code_configuration=code_config
)

In [81]:
ml_client.batch_deployments.begin_create_or_update(deployment).result()

Uploading batch_scoring_script (0.0 MBs): 100%|##########| 1040/1040 [00:00<00:00, 18162.23it/s]




BatchDeployment({'deployment_type': 'Model', 'job_definition': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'diabetes-78co6', 'type': None, 'name': 'diabetes-prediction', 'description': 'A diabetes classifier', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-workspace/batchEndpoints/diabetes-78co6/deployments/diabetes-prediction', 'Resource__source_path': None, 'base_path': 'd:\\AI\\MLOps\\EYMAY23\\MLOps-with-Azure', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000028A8A5AB0A0>, 'serialize': <msrest.serialization.Serializer object at 0x0000028A8A5ABD00>, 'model': '/subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-workspace/models/diabetes_model/versions/2', 'code_configuration': {'code': '/subscriptions/4980acc2-d47d-

In [82]:
endpoint = ml_client.batch_endpoints.get(endpoint.name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

In [83]:
print(f"The default deployment is {endpoint.defaults.deployment_name}")

The default deployment is diabetes-prediction


In [84]:
data_path = "diabetes-unlabeled.csv"
dataset_name = "unlabeled-diabetes"

dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FILE,
    description="An unlabeled dataset for diabetes",
    name=dataset_name,
)

In [85]:
ml_client.data.create_or_update(dataset_unlabeled)

Uploading diabetes-unlabeled.csv (< 1 MB): 100%|##########| 86.0/86.0 [00:00<00:00, 1.20kB/s]




Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'unlabeled-diabetes', 'description': 'An unlabeled dataset for diabetes', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-workspace/data/unlabeled-diabetes/versions/1', 'Resource__source_path': None, 'base_path': 'd:\\AI\\MLOps\\EYMAY23\\MLOps-with-Azure', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000028A8A537C40>, 'serialize': <msrest.serialization.Serializer object at 0x0000028A8A534A90>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/4980acc2-d47d-47e6-8498-a6b0744658d8/resourcegroups/EYMAY/workspaces/aml-workspace/datastores/workspaceblobstore/paths/LocalUpload/b431d23b18a17faae11c9366640b8c7f/diabetes-unlabeled.

In [86]:
dataset_unlabeled = ml_client.data.get(name=dataset_name, label="latest")

In [87]:
input = Input(type=AssetTypes.URI_FILE, path=dataset_unlabeled.id)

In [88]:
job = ml_client.batch_endpoints.invoke(endpoint_name=endpoint.name, input=input)

Exception: BY_POLICY